## Data Initializing

In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [2]:
import importlib
importlib.reload(dh)
importlib.reload(md)

<module 'model' from '/Users/kate/PycharmProjects/TransportNet-master/Stable Dynamic & Beckman/model.py'>

In [3]:
import numba
numba.__version__

'0.48.0'

# Stable Dynamics Model

parameter $\mu = 0$

In [11]:
net_name = 'Custom_net.tntp'
trips_name = 'Custom_trips.tntp'

sd_save = 'stable_dynamics_results/'
# net_name = 'Anaheim_net.tntp'
# trips_name = 'Anaheim_trips.tntp'

In [12]:
handler = dh.DataHandler()
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)
graph_data = handler.GetGraphData(net_name, 
                                  columns = ['init_node', 'term_node', 
                                             'capacity', 'free_flow_time'])
init_capacities = graph_data['graph_table']['capacity'] #* 2.5

## Base flows
First of all, we should find admissible set of flows on the transport graph. It is required for defining duality gap.

In [13]:
alpha = 0.75
# graph_data['graph_table']['capacity'] = init_capacities * alpha
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,2000.0,1.0
1,1,True,2,True,2000.0,0.5
2,2,True,1,True,2000.0,1.0
3,2,True,1,True,2000.0,0.5


In [14]:
graph_correspondences

{1: {'targets': [1, 2], 'corrs': [0.0, 3000.0]},
 2: {'targets': [2, 1], 'corrs': [0.0, 3000.0]}}

In [7]:
assert(model.mu == 0)
max_iter = 20000

solver_kwargs = {'eps': 1e-4,
                 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 200, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = '', composite = True,
                            solver_kwargs = solver_kwargs,
                            base_flows = alpha * graph_data['graph_table']['capacity'])
                            #base_flows here doesnt define anything now
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', 
      np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', 
      np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic

base_flows = result['flows']

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = 300

Iterations number: 200
Inner iterations number: 432
Primal_func_value = 3000
Dual_func_value = -4353.86
Duality_gap = -1053.86
Duality_gap / Duality_gap_init = -3.51288

Iterations number: 400
Inner iterations number: 832
Primal_func_value = 3000
Dual_func_value = -4353.32
Duality_gap = -1053.32
Duality_gap / Duality_gap_init = -3.51107

Iterations number: 600
Inner iterations number: 1232
Primal_func_value = 3000
Dual_func_value = -4352.78
Duality_gap = -1052.78
Duality_gap / Duality_gap_init = -3.50925

Iterations number: 800
Inner iterations number: 1632
Primal_func_value = 3000
Dual_func_value = -4352.23
Duality_gap = -1052.23
Duality_gap / Duality_gap_init = -3.50744

Iterations number: 1000
Inner iterations number: 2032
Primal_func_value = 3000
Dual_func_value = -4351.69
Duality_gap = -1051.69
Duality_gap / Duality_gap_init = -3.50562

It


Iterations number: 9600
Inner iterations number: 19232
Primal_func_value = 3000
Dual_func_value = -4327.5
Duality_gap = -1027.5
Duality_gap / Duality_gap_init = -3.425

Iterations number: 9800
Inner iterations number: 19632
Primal_func_value = 3000
Dual_func_value = -4326.92
Duality_gap = -1026.92
Duality_gap / Duality_gap_init = -3.42306

Iterations number: 10000
Inner iterations number: 20032
Primal_func_value = 3000
Dual_func_value = -4326.34
Duality_gap = -1026.34
Duality_gap / Duality_gap_init = -3.42113

Iterations number: 10200
Inner iterations number: 20432
Primal_func_value = 3000
Dual_func_value = -4325.76
Duality_gap = -1025.76
Duality_gap / Duality_gap_init = -3.41919

Iterations number: 10400
Inner iterations number: 20832
Primal_func_value = 3000
Dual_func_value = -4325.18
Duality_gap = -1025.18
Duality_gap / Duality_gap_init = -3.41726

Iterations number: 10600
Inner iterations number: 21232
Primal_func_value = 3000
Dual_func_value = -4324.59
Duality_gap = -1024.59
Dual


Iterations number: 19200
Inner iterations number: 38432
Primal_func_value = 3000
Dual_func_value = -4298.84
Duality_gap = -998.84
Duality_gap / Duality_gap_init = -3.32947

Iterations number: 19400
Inner iterations number: 38832
Primal_func_value = 3000
Dual_func_value = -4298.22
Duality_gap = -998.225
Duality_gap / Duality_gap_init = -3.32742

Iterations number: 19600
Inner iterations number: 39232
Primal_func_value = 3000
Dual_func_value = -4297.61
Duality_gap = -997.609
Duality_gap / Duality_gap_init = -3.32536

Iterations number: 19800
Inner iterations number: 39632
Primal_func_value = 3000
Dual_func_value = -4296.99
Duality_gap = -996.992
Duality_gap / Duality_gap_init = -3.32331

Iterations number: 20000
Inner iterations number: 40032
Primal_func_value = 3000
Dual_func_value = -4296.37
Duality_gap = -996.375
Duality_gap / Duality_gap_init = -3.32125

Result: success
Total iters: 20000
Primal_func_value = 3000
Dual_func_value = -4296.37
Duality_gap = -996.375
Duality_gap / Dualit

In [8]:
result.keys()

dict_keys(['times', 'flows', 'iter_num', 'res_msg', 'history', 'zone travel times', 'elapsed_time'])

In [9]:
result['zone travel times'], result['elapsed_time'], result['times']

({(1, 1): 0.0, (1, 2): 1.0, (2, 2): 0.0, (2, 1): 1.0},
 2758.261487007141,
 array([1.        , 1.06787503, 1.        , 1.06787503]))

In [26]:
base_flows 

array([  -0., 3000.,   -0., 3000.])

In [16]:
#with open(sd_save + 'anaheim_' + 
# 'ustm' + '_base_flows_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
#    pickle.dump(base_flows, f)

## Model

In [47]:
base_flows = np.array([150, 100, 100, 100])
# base_flows = result['flows']
print(type(result['flows']))
epsilons = np.logspace(2,0,9)
epsilons

<class 'numpy.ndarray'>


array([100.        ,  56.23413252,  31.6227766 ,  17.7827941 ,
        10.        ,   5.62341325,   3.16227766,   1.77827941,
         1.        ])

In [48]:
# graph_data['graph_table']['capacity'] = init_capacities
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,1500.0,1.0
1,1,True,2,True,1500.0,0.5
2,2,True,1,True,1500.0,1.0
3,2,True,1,True,1500.0,0.5


## Universal Similar Triangles

In [49]:
assert(model.mu == 0)
max_iter = 40000
for index, eps_abs in enumerate(epsilons):
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter, 'stop_crit': 'dual_gap',
                     'verbose': True, 'verbose_step': 2000, 'save_history': True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                    solver_kwargs = solver_kwargs, 
                                    base_flows = base_flows)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', 
          np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', 
          np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, 
          end = '\n\n')

    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
#     with open(sd_save + 'anaheim_result_' 
#               + 'ustm' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
#         pickle.dump(result, f)

eps_abs = 100.0
Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = -1394.74

Result: success
Total iters: 1
Primal_func_value = 3000
Dual_func_value = -3306.19
Duality_gap = -1700.92
Duality_gap / Duality_gap_init = 1.21953
Oracle elapsed time: 0 sec
Elapsed time: 0 sec
Time ratio = 1.2041241452319316
Flow excess = 1.0

eps_abs = 56.23413251903491
Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = -1394.74

Result: success
Total iters: 1
Primal_func_value = 3000
Dual_func_value = -3306.19
Duality_gap = -1700.92
Duality_gap / Duality_gap_init = 1.21953
Oracle elapsed time: 0 sec
Elapsed time: 0 sec
Time ratio = 1.2041241452319316
Flow excess = 1.0

eps_abs = 31.622776601683793
Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = 

In [51]:
result['flows']

array([  -0., 3000.,   -0., 3000.])

In [52]:
#index += 9
epsilons = [300, 200]

## Universal Gradient Descent

In [53]:
#weighted t added
assert(model.mu == 0)
max_iter = 40000
for index, eps_abs in enumerate(epsilons):
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter, 'stop_crit': 'dual_gap',
                     'verbose': True, 'verbose_step': 2000, 'save_history': True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ugd', composite = True,
                                    solver_kwargs = solver_kwargs, 
                                    base_flows = base_flows)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', 
          np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', 
          np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, 
          end = '\n\n')
    
    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
#     with open(sd_save + 'anaheim_result_' 
#         + 'ugd' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
#         pickle.dump(result, f)

eps_abs = 300
Composite optimization...
Oracles created...
Universal gradient descent method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = -1394.74

Result: success
Total iters: 1
Primal_func_value = 3000
Dual_func_value = -3306.19
Duality_gap = -1700.92
Duality_gap / Duality_gap_init = 1.21953
Oracle elapsed time: 0 sec
Elapsed time: 0 sec
Time ratio = 1.2041241452319316
Flow excess = 1.0

eps_abs = 200
Composite optimization...
Oracles created...
Universal gradient descent method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = -1394.74

Result: success
Total iters: 1
Primal_func_value = 3000
Dual_func_value = -3306.19
Duality_gap = -1700.92
Duality_gap / Duality_gap_init = 1.21953
Oracle elapsed time: 0 sec
Elapsed time: 0 sec
Time ratio = 1.2041241452319316
Flow excess = 1.0



In [54]:
result['flows']

array([  -0., 3000.,   -0., 3000.])

In [ ]:
#index += 9
#epsilons = [300, 200]

## Weighted Dual Averages

### Composite optimization

In [ ]:
assert(model.mu == 0)
max_iter = 50000
solver_kwargs = {'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 4000, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'wda', composite = True, 
                                solver_kwargs = solver_kwargs, base_flows = base_flows)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic
with open(sd_save + 'anaheim_result_' + 'wda' + '_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
    pickle.dump(result, f)

### Non-composite optimization

In [ ]:
assert(model.mu == 0)
max_iter = 50000
solver_kwargs = {'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 4000, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'wda', composite = False, 
                                solver_kwargs = solver_kwargs, base_flows = base_flows)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic
with open(sd_save + 'anaheim_result_' + 'wda_noncomposite' + '_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
    pickle.dump(result, f)

## Plots

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import pickle
import numpy as np
sd_save = 'stable_dynamics_results/'

In [ ]:
"""
for method in ['ustm', 'ugd']:
    results = []
    i = 0
    while True:
        try:
            with open(sd_save + 'anaheim_result_' + method + '_eps_abs_'
                      + str(i) + '_SD.pickle', 'rb') as f:
                result = pickle.load(f)
                results.append(result) 
        except FileNotFoundError:
            break
        i += 1
    results.sort(key = lambda res : res['eps_abs'])
    
    for i, result in enumerate(results):
        with open(sd_save + 'anaheim_result_' + method + '_eps_abs_' 
                  + str(i) + '_SD.pickle', 'wb') as f:
            pickle.dump(result, f)
"""

In [ ]:
results = {}

for method in ['ustm', 'ugd']:
    results[method] = {}
    results[method]['iters'] = []
    results[method]['inner_iters'] = []
    results[method]['epsilon'] = []
    results[method]['dual_gap_init'] = []
    i = 0
    while True:
        try:
            with open(sd_save + 'anaheim_result_' + method + '_eps_abs_'
                      + str(i) + '_SD.pickle', 'rb') as f:
                result = pickle.load(f)
                results[method]['epsilon'].append(result['eps_abs'])
                results[method]['iters'].append(result['iter_num'])
                results[method]['inner_iters'].append(result['history'].get('inner_iters')[-1])
                results[method]['dual_gap_init'].append(result['history'].get('dual_gap')[0])
        except FileNotFoundError:
            break
        i += 1

In [ ]:
for method in ['wda', 'wda_noncomposite']:
    results[method] = {}
    with open(sd_save + 'anaheim_result_' + method + '_max_iter_50000_SD.pickle', 'rb') as f:
        result = pickle.load(f)
        results[method]['epsilon'] = result['history'].get('dual_gap')
        results[method]['iters'] = result['history'].get('iter')
        results[method]['dual_gap_init'] = result['history'].get('dual_gap')[0]

In [ ]:
plt.figure(figsize=(7,4))
#plt.title('Convergence for Stable Dynamics model', fontsize=20)
for method, data in results.items():
    x = 1. / np.array(data['epsilon'])
    if 'inner_iters' in data:
        y = np.array(data['inner_iters'])
    else:
        y = np.array(data['iters'])
    plt.plot(x, y, label = method)
plt.legend(prop = {'size' : 11})
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'1 / $\varepsilon$', fontsize = 14)
plt.ylabel('inner iterations', fontsize = 14)
#plt.savefig('SD_convergence_1.jpg', dpi = 300)
plt.show()

In [ ]:
#inner iterations vs relative accuracy = eps_abs / dual_gap_init
plt.figure(figsize=(7,4))

method_names = {'ustm' : 'UMST', 'ugd' : 'UGM', 'wda' : 'WDA',
                'wda_noncomposite' : 'non-composite WDA'}
for method, data in results.items():
    x = 1. / (np.array(data['epsilon']) / np.array(data['dual_gap_init']))
    if 'inner_iters' in data:
        y = np.array(data['inner_iters'])
    else:
        y = np.array(data['iters'])
    plt.plot(x, y, label = method_names[method])
plt.legend(prop = {'size' : 11})
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'1 / $\tilde\varepsilon$', fontsize = 14)
plt.ylabel('inner iterations', fontsize = 14)
#plt.savefig('sd_convergence_rel_eps.jpg', dpi = 300)
plt.show()